In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib.font_manager import FontProperties
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# Graphics settings
plt.rcParams["figure.figsize"] = (3.2,3.2)
plt.rcParams['xtick.major.pad']='4'
plt.rcParams['ytick.major.pad']='4'
plt.rcParams['text.usetex'] = True
font = FontProperties()
font.set_name('serif')
font.set_size(8)
colours = np.array([(68,120,33), (141,211,95), (255,179,128), (255,221,85), (179,179,179)])/255
palette = sns.color_palette(colours)
palette2 = sns.dark_palette(colours[1], n_colors=4)
palette3 = sns.dark_palette(colours[2], n_colors=5,reverse=True)
sns.set_palette(palette)
sns.set_style("ticks")
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = '8'

os.makedirs('figures/', exist_ok=True)

# Urine rates
Data from BasicCatheter model

In [ ]:
# Location of results files
folder = '../raw_data/catheterResults/urine_rates/results'
files = os.listdir(folder)
files = sorted(files)

infection_threshold = np.array([1e0,1e2,1e4]) # Clinical definition of infection occuring

#  Threshold value for blockage
blockage_thresh = 1e7

# Arrays for results
N = len(files)
urine_rates = np.zeros(N)
times = np.zeros((N,len(infection_threshold)))
max_infect = np.zeros(N)
time_to_max = np.zeros(N)
time_to_block = np.zeros(N)

# Open files and read in data
for i in range(N):
    filename = os.path.join(folder, files[i])
    print(filename)
    # Read data
    df = pd.read_csv(filename, index_col=0, header=None, skiprows=3)
    info = pd.read_csv(filename,  nrows=1)
    if df.empty: break

    urine_rates[i] = info['urine rate'].iloc[0]*60/1000
    t_len = int(info['simulation length'].iloc[0]/info['print interval'].iloc[0]) # number of time steps
    dt = float(info['print interval'].iloc[0]/3600) # print interval (hrs)
    r=float(info['outside growth rate'].iloc[0])
    D=float(info['surface diffusivity'].iloc[0])
    L=float(info['catheter length'].iloc[0])
    fisher_time = 0.5*L/(np.sqrt(r*D)*3600*24)

    bladder = np.zeros(t_len)
    inside = np.zeros((t_len,df.shape[1]))
    outflow = np.zeros(t_len)
    maxy = np.zeros(t_len)
    for j in range(0, t_len):
        bladder[j] = df.iloc[4*j+1][1]
        inside[j] = df.iloc[4*j+2]
        outflow[j] = df.iloc[4*j+3][1]
        maxy[j] = max(inside[j])

    # Find times at which outflow crosses threshold
    indices = np.searchsorted(outflow,infection_threshold, side='right')
    times[i] = [dt*index if index!=len(outflow) else np.nan for index in indices]

    # Find maximum bladder infection & time to infection
    max_infect[i] = np.max(bladder)*1000
    time_to_max[i] = dt*np.searchsorted(bladder,0.99*np.max(bladder), side='right')

    # Find times at which inside blocks
    index = np.searchsorted(maxy,blockage_thresh, side='right')
    time_to_block[i] = dt*index if index!=t_len else np.nan

In [ ]:
plt.rcParams["figure.figsize"] = (3.2,3.2)
fig, main_ax = plt.subplots()

main_ax.scatter(urine_rates,max_infect,zorder=2,color=palette[1])
main_ax.plot(urine_rates,max_infect,zorder=2)
sns.despine()
main_ax.set_yscale('log')
main_ax.axvline(1, color=palette[2], ls='--',zorder=1, label='Typical\npatient\nflow rate')
#main_ax.legend(prop=font, bbox_to_anchor=(0.34, 0.21),handlelength=1,handletextpad=0.5,frameon=False)
plt.locator_params(axis='y',tight=True, numticks=5)
plt.locator_params(axis='x',tight=True, nbins=4)
main_ax.set_xlabel("Urine production rate (mL/min)", fontproperties=font, labelpad=2)
main_ax.set_ylabel("Steady state bacterial density (cells/mL)", fontproperties=font, labelpad=2)
main_ax.set_xlim(left=0)
main_ax.text(-0.39,1.5e9,'(a)')

sub_ax = fig.add_axes([0.739, 0.745, 0.25, 0.25])
sub_ax2 = fig.add_axes([0.739, 0.47, 0.25, 0.25], sharex=sub_ax)
sub_ax.tick_params(labelbottom=False)
sub_ax.plot(urine_rates, time_to_max/24,zorder=2)
sub_ax.axvline(1, color=palette[2], ls='--',zorder=1)
sub_ax.set_yticks([3,6,9])
sub_ax.set_ylim((2,10))
sub_ax.set_ylabel("Time (days)", fontproperties=font, labelpad=10,position=(0,0.53))

sub_ax2.plot(urine_rates,max_infect)
sub_ax2.axvline(1, color=palette[2], ls='--',zorder=1)
sub_ax2.set_yticks([0,1e9],['0','$10^9$'])
sub_ax2.set_ylim((-5e7,105e7))
sub_ax2.set_xlim(left=0)
sub_ax2.set_xticks([0,1,2])
sub_ax2.set_ylabel("Density (/mL)", fontproperties=font, labelpad=2)
sub_ax2.set_xlabel("Urine (mL/min)", fontproperties=font, labelpad=2,position=(0.48,0))

plt.tight_layout(rect=[-0.034,-0.036,1.033,1.032])
plt.savefig('figures/figure_3_a.pdf')

In [ ]:
plt.rcParams["figure.figsize"] = (1.9,1.9)
plt.figure()

plt.scatter(urine_rates[1::3],[time[0]/24 for time in times][1::3], label='10$^3$ mm$^{{-3}}$',s=5,color=palette2[1],zorder=2)
plt.scatter(urine_rates[1::3],[time[1]/24 for time in times][1::3], label='10$^5$ mm$^{{-3}}$',s=5,color=palette2[2],zorder=2)
plt.scatter(urine_rates[1::3],[time[2]/24 for time in times][1::3], label='10$^{7}$ mm$^{{-3}}$',s=5,color=palette2[3],zorder=2)
plt.axhline(fisher_time, color=palette[2], ls='--',zorder=1)
sns.despine()
plt.ylim(bottom=0)
plt.xlim(left=0)
plt.locator_params(tight=True, nbins=3)
plt.text(-0.5,4.3,'(a)')
plt.xlabel("Urine rate (mL/min)", fontproperties=font, labelpad=1.5)
plt.ylabel("Detection time (days)", fontproperties=font, labelpad=2)

plt.tight_layout(rect=[-0.06,-0.06,1.054,1.061]) #lbrt
plt.savefig('figures/figure_4_a.pdf')

# CDC NHANES data
Data on urine production rates in the US populations, from Centers for Disease Control and Prevention (CDC). National Center for Health Statistics (NCHS). National Health and Nutrition Examination Survey Data. Hyattsville, MD: U.S. Department of Health and Human Services, Centers for Disease Control and Prevention, 2009-2016. https://www.cdc.gov/nchs/nhanes/ [Accessed 6th March 2025].

In [ ]:
# Import data
path = '../raw_data/CDC_NHANES/'
data1 = pd.read_sas(path+'UCFLOW_F.XPT') # 2009-2010
demo1 = pd.read_sas(path+'DEMO_F.XPT')
data2 = pd.read_sas(path+'UCFLOW_G.XPT') # 2011-2012
demo2 = pd.read_sas(path+'DEMO_G.XPT')
data3 = pd.read_sas(path+'UCFLOW_H.XPT') # 2013-2014
demo3 = pd.read_sas(path+'DEMO_H.XPT')
data4 = pd.read_sas(path+'UCFLOW_I.XPT') # 2015-2016
demo4 = pd.read_sas(path+'DEMO_I.XPT')

In [ ]:
demo_df = pd.concat([demo.set_index('SEQN')[['RIAGENDR','RIDAGEYR','WTMEC2YR', 'SDMVPSU', 'SDMVSTRA']] for demo in [demo1,demo2,demo3,demo4]])
flow_df = pd.concat([data.set_index('SEQN')[['URDFLOW1','URDFLOW2','URDFLOW3']] for data in [data1,data2,data3,data4]])
mean_flow = flow_df.mean(axis=1).dropna() # Take mean flow rate and remove missing values
m_mean_flow = mean_flow[demo_df['RIAGENDR']==1][demo_df['RIDAGEYR']>18]
f_mean_flow = mean_flow[demo_df['RIAGENDR']==2][demo_df['RIDAGEYR']>18]
print(len(f_mean_flow),len(m_mean_flow)) # Check sample size
full_df = pd.concat([demo_df, mean_flow.to_frame("flow")],axis=1).dropna()
full_df['MEC8YR']=full_df['WTMEC2YR']/4 # Construct 8 year weights
print(full_df)

### Unweighted sample distribution

In [ ]:
plt.rcParams["figure.figsize"] = (2.3,3.2)
fig, (ax1, ax2) = plt.subplots(2,1, sharey=True, sharex=True)

ax1.hist(f_mean_flow,range=[0,5],bins=20)
ax2.hist(m_mean_flow,range=[0,5],bins=20)
plt.locator_params(nbins=4)
sns.despine()
ref_gr = 60*50*3.85e-4 # rV mL/min
ax1.axvline(ref_gr, color=palette[4], linestyle='--')
ax2.axvline(ref_gr, color=palette[4], linestyle='--')
ax1.set_ylabel('Frequency')
ax2.set_ylabel('Frequency')
ax1.text(-1.85,2800,'(b)')
ax1.set_title('Female', fontproperties=font, pad=-5)
ax2.set_title('Male', fontproperties=font, pad=-5)
ax2.set_xlabel('Urine production rate (mL/min)')

plt.tight_layout(rect=[-0.056,-0.037,1.045,1.035])
plt.savefig('figures/sex_disagg_hist_flow.pdf')
plt.show()

### Strip plot to show outliers

In [ ]:
plt.rcParams["figure.figsize"] = (5,3.2)
fig, ax = plt.subplots(1,2,sharex=True)

sns.stripplot(full_df[full_df['RIDAGEYR']>18],y='flow',s=1,x='RIAGENDR',ax=ax[0])
sns.stripplot(full_df[full_df['RIDAGEYR']>18],y='flow',s=1,x='RIAGENDR',ax=ax[1])
ax[1].set_ylim(0,10)

### Weighted resample

In [ ]:
m_data = full_df[full_df['RIAGENDR']==1][full_df['RIDAGEYR']>18]
m_sample = m_data.sample(n=100000,replace=True,weights='MEC8YR')
f_data = full_df[full_df['RIAGENDR']==2][full_df['RIDAGEYR']>18]
f_sample = f_data.sample(n=100000,replace=True,weights='MEC8YR')
data = pd.concat([f_sample['flow'].reset_index(drop=True),m_sample['flow'].reset_index(drop=True)],axis=1, keys=['Female','Male'])
print(data)

In [ ]:
# Check kde of weighted resample
sns.kdeplot(full_df[full_df['RIDAGEYR']>18],x='flow',hue='RIAGENDR',weights='MEC8YR',clip=[0,4],linestyle='--',common_norm=False,cut=0)
sns.kdeplot(full_df[full_df['RIDAGEYR']>18],x='flow',hue='RIAGENDR',clip=[0,4],common_norm=False,cut=0)
sns.kdeplot(data,clip=[0,4],bw_adjust=1.8,cut=0,linestyle=':',common_norm=False) # tune smoothing parameter

In [ ]:
# Check quantiles of weighted resample
print(m_data['flow'].quantile([.25,.5,.75])) # unweighted sample
print(m_sample['flow'].quantile([.25,.5,.75])) # weighted resample
mIQR=[0.556,0.877,1.408] # Quantiles calculated by Stata with weights
print(f_data['flow'].quantile([.25,.5,.75])) # unweighted sample
print(f_sample['flow'].quantile([.25,.5,.75])) # weighted resample
fIQR=[0.463,0.754,1.311] # Quantiles calculated by Stata with weights

In [ ]:
plt.rcParams["figure.figsize"] = (2.3,3.2)
fig, ax1 = plt.subplots()

sns.violinplot(data,cut=0,gridsize=1000,bw_adjust=1.8,common_norm=False) # Use full distribution for calculating kde & box-plot
plt.ylim(0,4) # Crop visible axis to region of interest
plt.locator_params(nbins=5)
ax1.set_ylabel('Urine production rate (mL/min)')
ax1.text(-0.88,4.3,'(b)')
ref_gr = 60*50*3.85e-4 # rV mL/min
ax1.axhline(ref_gr, color='gray', linestyle='--')
sns.despine()

plt.tight_layout(rect=[-0.04,-0.037,1.04,1.03])
plt.savefig('figures/figure_3_b.pdf')
plt.show()

print(sum(f_sample['flow']>4),sum(m_sample['flow']>4)) # Check how many points are in the cropped tails
print(sum(f_sample['flow']>4)/len(f_sample),sum(m_sample['flow']>4)/len(m_sample)) 
print(sum(f_mean_flow>4),sum(m_mean_flow>4))

# Figure 3

In [ ]:
plt.rcParams["figure.figsize"] = (5.5,3.2)
fig, (main_ax, bax) = plt.subplots(1,2,width_ratios=[3.2,2.3])

main_ax.scatter(urine_rates,max_infect,zorder=2,color=palette[1])
main_ax.plot(urine_rates,max_infect,zorder=2)
sns.despine()
main_ax.set_yscale('log')
main_ax.axvline(1, color=palette[2], ls='--',zorder=1, label='Typical\npatient\nflow rate')
#main_ax.legend(prop=font, bbox_to_anchor=(0.34, 0.21),handlelength=1,handletextpad=0.5,frameon=False)
main_ax.locator_params(axis='y',tight=True, numticks=5)
main_ax.locator_params(axis='x',tight=True, nbins=4)
main_ax.set_xlabel("Urine production rate (mL/min)", fontproperties=font, labelpad=2)
main_ax.set_ylabel("Steady state bacterial density (cells/mL)", fontproperties=font, labelpad=2)
main_ax.set_xlim(left=0)
fig.text(0,0.97,'(a)')

sub_ax = fig.add_axes([0.41, 0.745, 0.146, 0.25])
sub_ax2 = fig.add_axes([0.41, 0.47, 0.146, 0.25], sharex=sub_ax)
sub_ax.tick_params(labelbottom=False)
sub_ax.plot(urine_rates, time_to_max/24,zorder=2)
sub_ax.axvline(1, color=palette[2], ls='--',zorder=1)
sub_ax.set_yticks([3,6,9])
sub_ax.set_ylim((2,10))
sub_ax.set_ylabel("Time (days)", fontproperties=font, labelpad=10,position=(0,0.53))
sub_ax2.plot(urine_rates,max_infect)
sub_ax2.axvline(1, color=palette[2], ls='--',zorder=1)
sub_ax2.set_yticks([0,1e9],['0','$10^9$'])
sub_ax2.set_ylim((-5e7,105e7))
sub_ax2.set_xlim(left=0)
sub_ax2.set_xticks([0,1,2])
sub_ax2.set_ylabel("Density (/mL)", fontproperties=font, labelpad=2)
sub_ax2.set_xlabel("Urine (mL/min)", fontproperties=font, labelpad=2,position=(0.48,0))

sns.violinplot(data,cut=0, ax=bax,gridsize=1000,bw_adjust=1.8,common_norm=False)
bax.set_ylim(-0.2,4.2)
bax.locator_params(nbins=5)
bax.set_ylabel('Urine production rate (mL/min)')
fig.text(0.582,0.97,'(b)')
ref_gr = 60*50*3.85e-4 # rV mL/min
bax.axhline(ref_gr, color='gray', linestyle='--')

plt.tight_layout(rect=[-0.02,-0.034,1.018,1.0])
plt.savefig('figures/figure_3.pdf')

# Urethral lengths
Data from BasicCatheter model

In [ ]:
# Location of results files
folder = '../raw_data/catheterResults/urethral_lengths/results'
files = os.listdir(folder)
files = sorted(files)

# interested in time taken to detecting bacteriuria (at outflow) against catheter length
infection_threshold = np.array([1,1e2,1e4]) # Clinical definition of bacteriuria occuring

# Arrays for results
N = len(files)
lengths = np.zeros(N)
times2 = np.zeros((N,len(infection_threshold)))

for i in range(N):
    filename = os.path.join(folder, files[i])
    print(filename)
    # Read data
    df = pd.read_csv(filename, index_col=0, header=None, skiprows=3)
    info = pd.read_csv(filename,  nrows=1)
    if df.empty: break

    lengths[i] = info['catheter length'].iloc[0]
    t_len = int(info['simulation length'].iloc[0]/info['print interval'].iloc[0]) # number of time steps
    dt = float(info['print interval'].iloc[0]/3600) # print interval (hrs)
    r=float(info['outside growth rate'].iloc[0])
    D=float(info['surface diffusivity'].iloc[0])
    inverse_speed = 0.5/(np.sqrt(r*D)*3600*24)

    outflow = np.zeros(t_len)
    for j in range(0, t_len):
        outflow[j] = df.iloc[4*j+3][1]

    # Find times at which outflow crosses threshold
    indices = np.searchsorted(outflow,infection_threshold, side='right')
    times2[i] = [dt*index/24 if index!=len(outflow) else np.nan for index in indices]

In [ ]:
plt.rcParams["figure.figsize"] = (1.9,1.9)
plt.figure()

plt.scatter(lengths,[time[0] for time in times2], label='10$^3$ mm$^{{-3}}$',s=5,color=palette2[1],zorder=2)
plt.scatter(lengths,[time[1] for time in times2], label='10$^5$ mm$^{{-3}}$',s=5,color=palette2[2])
plt.scatter(lengths,[time[2] for time in times2], label='10$^7$ mm$^{{-3}}$',s=5,color=palette2[3])
plt.plot(lengths, lengths*inverse_speed, color=palette[2], ls='--',zorder=1)
sns.despine()
plt.ylim(bottom=0)
plt.locator_params(tight=True, nbins=3)
plt.xlabel('Urethral length (mm)', fontproperties=font, labelpad=2, position=(0.475,0))
plt.ylabel('Detection (days)', fontproperties=font, labelpad=2, position=(0,0.435))

plt.tight_layout(rect=[-0.072,-0.068,1.075,1.07])
plt.savefig('figures/figure_4_b.pdf')
plt.show()

# Figure 4

In [ ]:
plt.rcParams["figure.figsize"] = (4.0,1.9)
fig, (ax1,ax2) = plt.subplots(1,2,sharey=True)

ax1.scatter(urine_rates[1::3],[time[0]/24 for time in times][1::3], label='10$^3$ mm$^{{-3}}$',s=5,color=palette2[1],zorder=2)
ax1.scatter(urine_rates[1::3],[time[1]/24 for time in times][1::3], label='10$^5$ mm$^{{-3}}$',s=5,color=palette2[2],zorder=2)
ax1.scatter(urine_rates[1::3],[time[2]/24 for time in times][1::3], label='10$^{7}$ mm$^{{-3}}$',s=5,color=palette2[3],zorder=2)
ax1.axhline(fisher_time, color=palette[2], ls='--',zorder=1)

ax2.scatter(lengths,[time[0] for time in times2], label='10$^3$ mm$^{{-3}}$',s=5,color=palette2[1],zorder=2)
ax2.scatter(lengths,[time[1] for time in times2], label='10$^5$ mm$^{{-3}}$',s=5,color=palette2[2])
ax2.scatter(lengths,[time[2] for time in times2], label='10$^7$ mm$^{{-3}}$',s=5,color=palette2[3])
ax2.plot(lengths, lengths*inverse_speed, color=palette[2], ls='--',zorder=1)

sns.despine()
ax1.set_ylim(bottom=0)
ax1.set_xlim(left=0)
ax1.locator_params(tight=True, nbins=5)
ax1.text(-0.46,8.4,'(a)')
ax1.set_xlabel("Urine production rate (mL/min)", fontproperties=font, labelpad=2)
ax1.set_ylabel("Detection time (days)", fontproperties=font, labelpad=2)

ax2.set_ylim(bottom=0)
ax2.set_xlim(left=0)
ax2.text(-22,8.4,'(b)')
ax2.set_xlabel('Urethral length (mm)', fontproperties=font, labelpad=2)

plt.tight_layout(rect=[-0.028,-0.06,1.024,1.061]) #lbrt
plt.savefig('figures/figure_4.pdf')

# Figure 5
Data from BasicCatheter model

In [ ]:
plt.rcParams["figure.figsize"] = (4.0,4.2)
gs_kw = dict(width_ratios=[1,0.05,1], height_ratios=[1,1,0.1,1,1])
big_fig, ax = plt.subplots(5,3, sharex=True, sharey=True, gridspec_kw=gs_kw)

folder = '../raw_data/catheterResults/initial_conditions/'
files = ['results_skin.csv', 'results_bag.csv', 'results_uniform.csv', 'results_bladder.csv']

for f in range(4):
    file = folder + files[f]
    df = pd.read_csv(file, index_col=0, header=None, skiprows=3)
    info = pd.read_csv(file,  nrows=1)
    if df.empty: break
    t_len = int(info['simulation length'].iloc[0]/info['print interval'].iloc[0]) # number of time steps
    dt = float(info['print interval'].iloc[0]/3600) # print interval (hrs)
    time = dt*np.array(range(0, t_len)) # time series
    catheter_length = float(info['catheter length'].iloc[0])
    x_len = int(info['print num steps'].iloc[0]) # number of x steps
    dx = catheter_length/(x_len-1) # x step interval (mm)
    x = dx*np.array(range(0,x_len)) # x series
    outside = np.zeros((t_len,df.shape[1]))
    inside = np.zeros((t_len,df.shape[1]))
    for i in range(0, t_len):
        outside[i] = df.iloc[4*i]
        inside[i] = df.iloc[4*i+2]
    outside = outside*1e-7
    inside = inside*1e-7
    maxx = max(max(outside[-1]),max(inside[-1]))
    index24 = int(24/dt) #find index for 24 hrs
    inside_1 = inside[int(index24*1)][::-1]
    outside_1 = outside[int(index24*1)]
    inside_2 = inside[int(index24*1.5)][::-1]
    outside_2 = outside[int(index24*1.5)]

    i = (f//2)*3
    j = f%2*2
    
    ax[i+1,j].plot(x,inside_1,color=palette[2],linestyle='dashed',linewidth=1.5)
    ax[i,j].plot(x,outside_1,color=palette[1],linestyle='dashed',linewidth=1.5)
    ax[i+1,j].plot(x,inside_2,color=palette3[2],linewidth=2)
    ax[i,j].plot(x,outside_2,color=palette[0],linewidth=2)
    sns.despine()
    ax[i,j].set_ylim(0,1.035*maxx)
    ax[i+1,j].set_ylim(0,1.035*maxx)
    ax[i,j].set_yticks([0,0.5,1],['0','5','10'])
    ax[i+1,j].set_yticks([0,0.5,1],['0','5', '10'])
    ax[i,j].tick_params(axis='y',pad=2)
    ax[i+1,j].tick_params(axis='y',pad=2)


ax[2,0].axis('off')
ax[2,1].axis('off')
ax[2,2].axis('off')
ax[0,1].axis('off')
ax[1,1].axis('off')
ax[3,1].axis('off')
ax[4,1].axis('off')

big_fig.add_subplot(111,frameon=False)
plt.tick_params(labelcolor='none',which='both',top=False,bottom=False,left=False,right=False)
plt.ylabel('Bacteria per surface area ($10^6$~mm$^{-2}$)', labelpad=2, fontproperties=font)
plt.xlabel('Distance up urethra (mm)', labelpad=1.5, fontproperties=font)
plt.text(0.005,1.02, '(a) Skin', fontproperties=font)
plt.text(0.565,1.02, '(b) Drainage bag', fontproperties=font)
plt.text(0.005,0.46, '(c) Insertion', fontproperties=font)
plt.text(0.565,0.46, '(d) Bladder', fontproperties=font)
#plt.text(1.04,0.9, 'Ext.', fontproperties=font, rotation='vertical', va='center')
#plt.text(1.04,0.65, 'Int.', fontproperties=font, rotation='vertical', va='center')
#plt.text(1.04,0.33, 'Ext.', fontproperties=font, rotation='vertical', va='center')
#plt.text(1.04,0.08, 'Int.', fontproperties=font, rotation='vertical', va='center')
plt.tight_layout(rect=[-0.073,-0.076,1.045,1.025])
plt.savefig("figures/figure_5.pdf")




# Outside surface
Simulate behaviour of FKPP equation without coupling to the rest of the catheter model.

In [ ]:
def init(iT=24, iD=1e-4, ino=100, iL=40):
    global D,r,kappa,T,dt,Nt,L,model_length,N,dx,skin_conc,const1,const2,const3,conc,old_conc
    D = iD # diffusivity (mm^2/s)
    r = np.log(2)/(3600) # Growth rate (s^-1)
    kappa = 1e6 # Carrying capacity
    T = iT * 60 * 60 # Timescale (s)
    L = iL # Catheter length (mm)
    model_length = L+200
    dx_rough = 0.5*np.sqrt(D/r)
    N = int(model_length/dx_rough)*2 + 1 # Number of x steps
    dx = model_length/(N-1) # x step (mm)
    dt = 0.05*dx*dx/D # timestep (s)
    Nt = int(np.around(T/dt,-2))
    dt = T/Nt
    skin_conc = ino # Concentration of bacteria on skin (mm^-2), i.e. the boundary condition
    const1 = D*dt/(dx*dx)
    const2 = 1-2*const1+r*dt
    const3 = r*dt/kappa
    conc = np.zeros(N)
    old_conc = np.zeros_like(conc)
    index = int(100/dx)
    conc[index] = skin_conc/dx
    old_conc[index] = skin_conc
    print(N, dx, Nt, dt)

def step():
    global conc,old_conc
    for i in range(1,N-1):
        conc[i] = const1*(old_conc[i+1]+old_conc[i-1]) + old_conc[i]*(const2-const3*old_conc[i])
    #boundaries
    conc[0] = 2*const1*(old_conc[1]) + old_conc[0]*(const2-const3*old_conc[0])
    conc[N-1] = 2*const1*(old_conc[N-2]) + old_conc[N-1]*(const2-const3*old_conc[N-1])
    conc, old_conc = old_conc, conc

In [ ]:
init(iT=72)
saves = []
for l in range(12):
    for s in range(int(Nt/12)):
        step()
    saves.append(old_conc.copy())

In [ ]:
plt.rcParams["figure.figsize"] = (2.0,1.55)
fig,ax = plt.subplots()

display_val = int(L/dx)
start_val = int(100/dx)
xvals = np.linspace(0,L,display_val)

plt.plot(xvals,saves[3][start_val:display_val+start_val]*1e-5, label='24 hrs', color=palette3[0])
plt.plot(xvals,saves[5][start_val:display_val+start_val]*1e-5, label='36 hrs', color=palette3[1])
plt.plot(xvals,saves[7][start_val:display_val+start_val]*1e-5, label='48 hrs', color=palette3[2])
sns.despine()
plt.xlim((0,40))
plt.ylim(bottom=0)
plt.locator_params(nbins=3)
plt.xlabel('Distance up urethra (mm)',fontproperties=font,labelpad=2,position=(0.34,0))
plt.ylabel('Bacteria/area ($10^6$ mm$^{-2}$)',fontproperties=font,labelpad=2,position=(0,0.33))
ax.yaxis.set_label_coords(-0.16,0.42)
#plt.legend(prop=font, bbox_to_anchor=(1.19, -0.46),  ncol=2,columnspacing=2,handlelength=2,handletextpad=0.5)

plt.tight_layout(rect=[-0.055,-0.07,1.057,1.07])
plt.savefig('figures/figure_2_b.pdf')
plt.show()

# Bladder
Simulate behaviour of logistic growth with dilution equation without coupling to the rest of the catheter model.

In [ ]:
def init(ik_D = 10/3, irho=1, iT=96 ):
    global r, kappa, k_D, T, dt, N, rho, Nt, outtime, times, conc
    r = np.log(2)/(1800) # Growth rate (s^-1)
    kappa = 1e6 # Carrying capacity
    k_D = ik_D*1e-4 # Dilution rate (s^-1)
    T = iT * 60 * 60 #Timescale (s)
    dt = 1 #timestep(s)
    N = 200 #Number of output steps
    rho = irho #Concentration
    Nt = int(T/dt) #Number of time steps
    outtime = int(Nt/N) #Time step at which to output
    times = np.linspace(0,iT,N)

def step(rho):
    change = r*rho*(1-rho/kappa) - k_D*rho
    return rho + dt*change

In [ ]:
init(ik_D=0,iT=48)
conc0 = np.zeros_like(times)
for i in range(N):
    conc0[i] = rho
    for j in range(outtime):
        rho = step(rho)
init(ik_D=1,iT=48)
conc1 = np.zeros_like(times)
for i in range(N):
    conc1[i] = rho
    for j in range(outtime):
        rho = step(rho)
init(ik_D=2,iT=48)
conc2 = np.zeros_like(times)
for i in range(N):
    conc2[i] = rho
    for j in range(outtime):
        rho = step(rho)
init(ik_D=3,iT=48)
conc3 = np.zeros_like(times)
for i in range(N):
    conc3[i] = rho
    for j in range(outtime):
        rho = step(rho)

In [ ]:
plt.rcParams["figure.figsize"] = (2.0,1.55)

plt.plot(times/24,conc0*1e-5,linewidth=2,label=r'0$\times10^{-4}$s$^{-1}$', color=palette2[0])
plt.plot(times/24,conc1*1e-5,linewidth=2,label=r'1$\times10^{-4}$s$^{-1}$', color=palette2[1])
plt.plot(times/24,conc2*1e-5,linewidth=2,label=r'2$\times10^{-4}$s$^{-1}$', color=palette2[2])
plt.plot(times/24,conc3*1e-5,linewidth=2,label=r'3$\times10^{-4}$s$^{-1}$', color=palette2[3])
plt.ylim(0,10.2)
sns.despine()
plt.xlim(0,2)
plt.locator_params(nbins=3)
plt.locator_params(axis='x',tight=True)
plt.ylabel("Bacteria/volume ($10^8$/mL)",fontproperties=font, labelpad=1.5)#, position=(0,0.57))
plt.xlabel("Time (days)",fontproperties=font,labelpad=2)

#plt.text(3, -8.7,r'$   k_D$' '\n' r'($10^{-5}$ s$^{-1}$)',fontproperties=font,ha='right')
#plt.legend(prop=font, bbox_to_anchor=(1.11, -0.38), ncol=2,columnspacing=0.75,handlelength=1,handletextpad=0.5)

plt.tight_layout(rect=[-0.057,-0.074,1.06,1.058])
plt.savefig('figures/figure_2_e.pdf')
plt.show()

# Luminal flow
Simulating behaviour of advection-diffusion equation without coupling to the rest of the catheter model.

In [ ]:
def init(*,iT=30,iNt=3000,iout_time=0.5,icoarsening=2,iR=1,iL=100,iNr=21,iNx=21,iD=1e-4,iflow_rate=100/6,irho=1e6):
    '''Modify parameters of problem'''
    global T,Nt,out_time,coarsening,R,L,Nr,Nx,D,flow_rate,rho,dt,dr,dx,Nout,conc,new_conc,surface,flow_time
    T,Nt,out_time,coarsening,R,L,Nr,Nx,D,flow_rate,rho = iT,iNt,iout_time,icoarsening,iR,iL,iNr,iNx,iD,iflow_rate,irho
    dt = T/Nt # simulation time step
    dr = R/(Nr-1) # simulation radial step
    dx = L/(Nx-1) # simulation length step
    Nout = int(out_time/dt) # number of time steps between each output
    conc = np.zeros((Nx,Nr)) # Bacterial concentration within the flow (mm^-3) indexed [i,j] as length and radius respectively
    new_conc = np.zeros((Nx,Nr)) # Bacterial concentration within the flow (mm^-3) at the next time step
    surface = np.zeros(Nx) # Flux through the catheter wall in current time step (mm^-2 s^-1)
    flow_time = (np.pi*R**2)/flow_rate # Time taken on average for fluid to travel 1 mm (s)
    

In [ ]:
def step():
    '''Solve the PDE problem for one time-step'''
    c1 = D*dt/(dr*dr)
    c2 = 2*flow_rate*dt/(np.pi*R**4*dx)
    c3 = 1-2*c1
    c7b = c2*R*R
    c8b = c3-c7b
    global conc
    global new_conc
    global surface
    # Main loop
    # Loop over radius
    for j in range(1,Nr-1):
        c4 = 0.5/j
        c5 = c1*(1+c4)
        c6 = c1*(1-c4)
        c7 = c2*(R*R-j*j*dr*dr)
        c8 = c3-c7
        # Loop over length
        for i in range(1,Nx):
            new_conc[i][j] = c5*conc[i][j+1]+c6*conc[i][j-1]+c7*conc[i-1][j]+c8*conc[i][j]
        # boundary condition i=0 (top of catheter)
        new_conc[0][j] = c5*conc[0][j+1]+c6*conc[0][j-1]+c7*rho+c8*conc[0][j]
    # boundaries
    # Boundary condition j=Nr-1, do nothing (keep at zero)
    # Boundary condition i=j=0 (top centre)
    new_conc[0][0] = 2*c1*conc[0][1]+c7b*rho+c8b*conc[0][0]
    for i in range(1,Nx):
        # Boundary condition j=0 (centre of catheter) 
        new_conc[i][0] = 2*c1*conc[i][1]+c7b*conc[i-1][0]+c8b*conc[i][0]
        # Update surface
        surface[i] = D*conc[i][Nr-2]/dr
    # Update surface at i=0
    surface[0] = D*conc[0][Nr-2]/dr
    # Update arrays
    conc, new_conc = new_conc, conc
    # Sanity check 
    #if np.max(abs(conc-new_conc))>10*rho:
    #    raise ValueError("Solution diverging")
            

In [ ]:
init(irho=100,iNr=51,iNx=51,iNt=6000, iR=1, iL=40, iD=1e-4, iT=300)
for s in range(Nt):
    step()

In [ ]:
plt.rcParams["figure.figsize"] = (2.0,1.55)
    
cmappy = LinearSegmentedColormap.from_list("greenyellow",[(0,palette[4]),(0,palette[3]),(1,palette[0])],N=256)

rlist = np.linspace(0,R,Nr)
walls = np.zeros((Nx,int(Nr/4)))
complete = np.concatenate([conc,walls],axis=1)

axy=sns.heatmap(complete*0.1,cmap=cmappy,cbar_kws={"ticks":[0,5,10]})

plt.xticks([0,Nr],[0,R],rotation=0)
plt.yticks([0,Nx/2,Nx],[0,int(L/2),L])
sns.despine(ax=axy,offset=3,trim=True)
plt.ylabel('Distance down lumen (mm)',fontproperties=font,labelpad=2)
plt.xlabel('Radial distance (mm)',fontproperties=font,labelpad=2)
plt.text(88,60,'Bacteria/volume ($10^8$/mL)',rotation=90,fontproperties=font)

plt.tight_layout(rect=[-0.057,-0.07,0.98,1.07])
plt.savefig('figures/figure_2_h.pdf')
plt.savefig('figures/figure_2_h.png',dpi=400)
plt.show()

### Supplementary luminal flow

In [ ]:
def flow(r):
    v = 2*flow_rate/(np.pi*R**4)*(R**2-r**2)
    return v*(v>0)

r_arr = np.linspace(-R,R,15)
y_arr = np.zeros_like(r_arr)
y_dir = -1*flow(r_arr)
r_dir = np.zeros_like(y_dir)

plt.quiver(r_arr,y_arr,r_dir,y_dir,scale_units = 'xy',scale=1,color=palette[3],width=0.015)
plt.locator_params(tight=True, nbins=4)
plt.xticks([0,R],[0,R])
plt.ylim(0,11)
plt.xlim(-1.5*R,1.5*R)
plt.gca().invert_yaxis()
plt.fill([1,1,1.5,1.5],[0,11,11,0],[-1,-1,-1.5,-1.5],[0,11,11,0],color=palette[4])
sns.despine(offset=10, trim=True)
plt.ylabel(r'Flow velocity, $u$ (mm s$^{-1}$)',fontproperties=font,labelpad=2,position=(0,0.49))
plt.xlabel(r'Radial distance, $r$ (mm)',fontproperties=font,labelpad=2,position=(0.5,0))

plt.tight_layout(rect=[-0.065,-0.07,1.06,1.07])
plt.savefig('figures/flow_profile.pdf')
plt.show()

In [ ]:
# Theoretical solution
def levich(x):
    return 0.5835*D*rho*np.cbrt(flow_rate/(R*R*R*D))/np.cbrt(x)

plt.rcParams["figure.figsize"] = (2.5,2.5)

x = np.linspace(0,L,Nx)

plt.plot(x,levich(x)*10,color=palette[1], linewidth=2, label='Analytic')
plt.plot(x,surface*10,color=palette[0], linewidth=2, label='Numeric')
sns.despine()
plt.ylim(0,4)
plt.xlim(0,40)
plt.locator_params(nbins=3)
plt.ylabel(r'Bacterial flux, $j$ ($10^3$ mm$^{-2}$s$^{-1}$)',fontproperties=font,labelpad=2,position=(0,0.435))
plt.xlabel(r'Distance down inside, $x$ (mm)',fontproperties=font,labelpad=2,position=(0.465,0))
plt.legend(prop=font, bbox_to_anchor=(1.1, 1.0), handlelength=1.2,handletextpad=0.5) 

plt.tight_layout(rect=[-0.066,-0.07,1.07,1.07])
plt.savefig('figures/bacterial_flux.pdf')
plt.show()